## Face expression Recognition

* import os and get the current working directory

In [1]:
import os
os.getcwd()

'C:\\Users\\saite\\Desktop\\face reco'

* Let's assign the dataset path to the basepath

In [2]:
base_path = "C:\\Users\\saite\\Desktop\\face reco\\images\\images"

* Get the number of images of each class in train dataset

In [3]:
# count number of train images for each expression

for expression in os.listdir(base_path + "\\train"):
    print(str(len(os.listdir(base_path + "\\train\\" + expression))) + " " + expression + " images")

3988 angry images
436 disgust images
4103 fear images
7164 happy images
4982 neutral images
4938 sad images
3205 surprise images


* we have good amount of images for each class except for the class disgust. 

* pic_size for the images is (48,48)

In [20]:
pic_size = 48

#### Train Validation Generator
* The color of the images are gray
* consider the batch size 128
* Shuffle the data for each generator

In [5]:
from keras.preprocessing.image import ImageDataGenerator

datagen_train = ImageDataGenerator()
datagen_validation = ImageDataGenerator()

train_generator = datagen_train.flow_from_directory(base_path + "\\train",
                                                    target_size=(pic_size,pic_size),
                                                    color_mode="grayscale",
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    shuffle=True)

validation_generator = datagen_validation.flow_from_directory(base_path + "\\validation",
                                                    target_size=(pic_size,pic_size),
                                                    color_mode="grayscale",
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    shuffle=True)


Using TensorFlow backend.


Found 28816 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


### Import the basic Library's

In [7]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import tensorflow as tf

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

### Building Neural Network
* I have added 4 convolution layers followed by maxpooling 
* Increasing the number of channels from 1(input) to 512
* input is zero padded with (3,3) 
* (3,3) filter for each convolution layer
* Finally one fully connected layer with sigmoid activation function

In [8]:
input_shape = (48,48,1)
X_input = Input(input_shape)

# Zero-Padding: pads the border of X_input with zeroes
X = ZeroPadding2D((3, 3))(X_input)

# CONV -> BN -> RELU Block applied to X
X = Conv2D(16, (3, 3), strides = (1, 1), name = 'conv0')(X)
print(X)
X = BatchNormalization(axis = 3, name = 'bn0')(X)
X = Activation('relu')(X)

# MAXPOOL
X = MaxPooling2D((2, 2), name='max_pool0')(X)

# CONV -> BN -> RELU Block applied to X
X = Conv2D(64, (3, 3) , strides = (1, 1), name = 'conv1')(X)
X = BatchNormalization(axis = 3, name = 'bn1')(X)
X = Activation('relu')(X)

# MAXPOOL
X = MaxPooling2D((2, 2), name='max_pool1')(X)

# CONV -> BN -> RELU Block applied to X
X = Conv2D(128, (3, 3), strides = (1, 1), name = 'conv2')(X)
X = BatchNormalization(axis = 3, name = 'bn2')(X)
X = Activation('relu')(X)

# MAXPOOL
X = MaxPooling2D((2, 2), name='max_pool2')(X)

# CONV -> BN -> RELU Block applied to X
X = Conv2D(512, (3 ,3), strides = (1, 1), name = 'conv3')(X)
X = BatchNormalization(axis = 3, name = 'bn3')(X)
X = Activation('relu')(X)

# MAXPOOL
X = MaxPooling2D((2, 2), name='max_pool3')(X)



# FLATTEN X (means convert it to a vector) + FULLYCONNECTED
X = Flatten()(X)
X = Dense(1, activation='sigmoid', name='fc')(X)

# Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
model = Model(inputs = X_input, outputs = X)

Tensor("conv0/BiasAdd:0", shape=(None, 52, 52, 16), dtype=float32)


#### Compiling The Model
* Adam Optimizer with 0.01 as learning rate
* categorical cross entropy is used as loss function

In [9]:
model.compile(optimizer=Adam(lr = 0.01), loss= tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])


#  Run the Model

In [10]:
%%time

model.fit_generator(generator=train_generator,
                                steps_per_epoch=train_generator.n//train_generator.batch_size,
                                epochs=50,
                                validation_data = validation_generator,
                                validation_steps = validation_generator.n//validation_generator.batch_size,
                                )

Epoch 1/50
225/225 [==============================] - 748s 3s/step - loss: 1.1921e-07 - accuracy: 0.5003 - val_loss: 1.1921e-07 - val_accuracy: 0.4011
Epoch 2/50
225/225 [==============================] - 238s 1s/step - loss: 1.1921e-07 - accuracy: 0.5008 - val_loss: 1.1921e-07 - val_accuracy: 0.4906
Epoch 3/50
225/225 [==============================] - 239s 1s/step - loss: 1.1921e-07 - accuracy: 0.5000 - val_loss: 1.1921e-07 - val_accuracy: 0.4950
Epoch 4/50
225/225 [==============================] - 236s 1s/step - loss: 1.1921e-07 - accuracy: 0.5004 - val_loss: 1.1921e-07 - val_accuracy: 0.4963
Epoch 5/50
225/225 [==============================] - 236s 1s/step - loss: 1.1921e-07 - accuracy: 0.5000 - val_loss: 1.1921e-07 - val_accuracy: 0.4957
Epoch 6/50
225/225 [==============================] - 236s 1s/step - loss: 1.1921e-07 - accuracy: 0.4999 - val_loss: 1.1921e-07 - val_accuracy: 0.4976
Epoch 7/50
225/225 [==============================] - 236s 1s/step - loss: 1.1921e-07 - accura

* We have got an accuracy of 50% for 50 epochs
* I have run this model only with CPU and no graphic card so the time taken is nearly 3 hours
* Approximately 17 minutes to run the same model with 50 eopchs in kaggle with GPU enabled


* Let's Save the model for future reference

In [11]:
model.save('model.h5')

### Accuracy 
 #### Accuracy can be increased by  
 * changing number of epochs
 * changing number of layers in CNN
 * Changing number of channels
 * Changing activation functions
 * Tuning the remaining hyperparameters

check my work on [kaggle](./https://www.kaggle.com/saitej31) by clicking here.

#### Dataset is taken from kaggle